In [3]:
from pprint import pprint
import pandas as pd
import numpy as np
import redefine

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [4]:
data = pd.read_csv('data/iris_true.csv')

In [5]:
data

,id,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,virginica
146,146,6.3,2.5,5.0,1.9,virginica
147,147,6.5,3.0,5.2,2.0,virginica
148,148,6.2,3.4,5.4,2.3,virginica


In [5]:
rd = redefine.REDEFINE('iris_true.csv', data, 'target', 'id')

In [6]:
results = rd.run_redefine('Random Forest', {'n_estimators':'5'}, 'KMeans', {}, 'Standard')

In [ ]:
results.to_csv('testing.txt', sep='\t')

In [ ]:
X = rd.get_X()
IDs = rd.get_IDs()
Y = rd.get_Y()
Y_names = rd.get_Y_names()
kfolds = 10